In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

In [2]:
import math

In [3]:
folder = '/media/misha/Data/Game Captures GTA/'

In [4]:
cap = cv2.VideoCapture(folder + '0-country-test.mov')

In [5]:
fgbg = cv2.createBackgroundSubtractorMOG2()
kernel = np.ones((3,3),np.uint8)

### Pipeline
1. Preproces:
    - Convert to Gray
    - Downscale
    - Blur
2. BGS using GMM
3. Find moving objects and track
4. Extract movement features
5. Classify them

Question: how to initialize mean-shift tracking

see: Vehicle tracking by non-drifting mean-shift using projective Kalman filter https://pdfs.semanticscholar.org/c4b0/18193f813eaa8981ee4e85657669e5865fab.pdf

### BGS GMM + Morph_open

In [9]:
r,h,c,w = 100,90,200,125  # simply hardcoded the values
track_window = (c,r,w,h)

# set up the ROI for tracking
roi = frame[r:r+h, c:c+w]
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 3 )

In [6]:
# Setup the termination criteria, either 10 iteration or move by atleast 1 
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 3 )

In [7]:
def preprocessFrame(frame):
#     frame = cv2.resize(frame, (640, 360), interpolation=cv2.INTER_AREA)
    frame_bw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_bw = cv2.GaussianBlur(frame_bw, (21, 21), 0)
    return frame_bw

#### Test begin:

In [8]:
cap = cv2.VideoCapture(folder + '0-country-test.mov')

In [11]:
ret, frame = cap.read()
frame = cv2.resize(frame, (640, 360), interpolation=cv2.INTER_AREA)
frame_bw = preprocessFrame(frame)
fgmask = fgbg.apply(frame_bw)
flt_fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
flt_fgmask = cv2.threshold(flt_fgmask, 25, 255, cv2.THRESH_BINARY)[1]
flt_fgmask = cv2.dilate(flt_fgmask, None, iterations=2)

In [12]:
hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
cnts = cv2.findContours(flt_fgmask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]

In [13]:
super_old_cnts0 = []
old_cnts0 = []

super_old_cnts1 = []
old_cnts1 = []

In [18]:
super_old_cnts0 = [None]*len(cnts)
old_cnts0 = [None]*len(cnts)
super_old_cnts0[0] = cv2.boundingRect(cnts[0])

In [19]:
for i in range(len(super_old_cnts0)):
    print(i)

0


In [15]:
x,y,w,h

NameError: name 'x' is not defined

In [34]:
(x,y,w,h) = super_old_cnts0[0]

In [47]:
super_old_cnts0[0]

(438, 94, 21, 20)

In [49]:
(x,y,w,h) = super_old_cnts0[0]
roi = frame[y:y+h, x:x+w]
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
ret, track_window = cv2.meanShift(dst, (x,y,w,h), term_crit)
old_cnts0[0] = track_window

IndexError: list assignment index out of range

In [44]:
# (x, y, w, h)
super_old_cnts0[0]

array([[[448,  94]],

       [[447,  95]],

       [[446,  95]],

       [[445,  96]],

       [[444,  96]],

       [[442,  98]],

       [[441,  98]],

       [[441,  99]],

       [[440, 100]],

       [[440, 101]],

       [[438, 103]],

       [[438, 109]],

       [[439, 110]],

       [[440, 110]],

       [[442, 112]],

       [[444, 112]],

       [[445, 113]],

       [[454, 113]],

       [[456, 111]],

       [[456, 110]],

       [[457, 109]],

       [[457, 104]],

       [[458, 103]],

       [[458,  97]],

       [[457,  96]],

       [[457,  95]],

       [[456,  94]]], dtype=int32)

In [ ]:
plt.imshow(roi)

#### Test end:

In [24]:
# ret, frame = cap.read()
# fgmask = fgbg.apply(frame)
cap = cv2.VideoCapture(folder + '0-country-test.mov')
fr_number = 0
# super_old_cnts0 = []
# old_cnts0 = []

# super_old_cnts1 = []
# old_cnts1 = []

while(1):
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 360), interpolation=cv2.INTER_AREA)
    frame_bw = preprocessFrame(frame)
    fgmask = fgbg.apply(frame_bw)
    flt_fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    flt_fgmask = cv2.threshold(flt_fgmask, 25, 255, cv2.THRESH_BINARY)[1]
    flt_fgmask = cv2.dilate(flt_fgmask, None, iterations=2)
    

#     track some window...
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    
#     ret, track_window = cv2.meanShift(dst, track_window, term_crit)
#     x,y,w,h = track_window
#     img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)
    
#     ret, track_window2 = cv2.meanShift(dst, (x2, y2, w2, h2), term_crit)
#     x2,y2,w2,h2 = track_window2
#     cv2.rectangle(img2, (x2, y2), (x2 + w2, y2 + h2), (255, 255, 0), 2)
    
    cnts = cv2.findContours(flt_fgmask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]
    
    # Now we have new good contours and need to run tracking:
    
    if fr_number % 2 == 0:
        # calc for the third frame:
        for i in range(len(old_cnts0)):
            if old_cnts0[0] == None: continue
            (x,y,w,h) = super_old_cnts0[i]
            roi = frame[y:y+h, x:x+w]
            hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
            roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
            cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
            dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
            ret, track_window = cv2.meanShift(dst, (x,y,w,h), term_crit)
            
        
            # calc acc and speed
            print(super_old_cnts0[i], old_cnts0[i], track_window, 
                  math.sqrt((old_cnts0[i][0] - track_window[0])**2 + (old_cnts0[i][1] - track_window[1])**2),
                 math.sqrt((super_old_cnts0[i][0] - old_cnts0[i][0])**2 + (super_old_cnts0[i][1] - old_cnts0[i][1])**2))
        # make all containers null
        super_old_cnts0 = [None]*len(cnts)
        old_cnts0 = [None]*len(cnts)
        # record new frame
        for i in range(len(cnts)):
            super_old_cnts0[i] = cv2.boundingRect(cnts[i])
        
        # update frames for alternative thread
        for i in range(len(super_old_cnts1)):
            (x,y,w,h) = super_old_cnts1[i]
            roi = frame[y:y+h, x:x+w]
            hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
            roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
            cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
            dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
            ret, track_window = cv2.meanShift(dst, (x,y,w,h), term_crit)
            
            old_cnts1[i] = track_window
        
        
    else: # Fix me
        super_old_cnts1 = cnts
        # update second frame
        for i in range(len(super_old_cnts0)):
            
            (x,y,w,h) = super_old_cnts0[i]
            roi = frame[y:y+h, x:x+w]
            hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
            roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
            cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

            dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
            ret, track_window = cv2.meanShift(dst, super_old_cnts0[i], term_crit)
            old_cnts0[i] = track_window
    
        # calculate third value and classify
    
    
#     for i in range(len(old_cnts)):
#         ret, track_window = cv2.meanShift(dst, old_cnts[i], term_crit)
#         print(old_cnts[i], track_window, 
#               math.sqrt((old_cnts[i][0] - track_window[0])**2 + (old_cnts[i][1] - track_window[1])**2),
#              math.sqrt((super_old_cnts[i][0] - old_cnts[i][0])**2 + (super_old_cnts[i][1] - old_cnts[i][1])**2))
    
#     for i in range(len(cnts)):
#         old_cnts.append(cv2.boundingRect(cnts[i]))
#         x,y,w,h = old_cnts[i]
#         cv2.rectangle(img2, (x,y), (x+w,y+h), (0, 255, i), 2)
    
    
    for c in cnts:
        if cv2.contourArea(c) < 50:
            continue

            # compute the bounding box for the contour, draw it on the frame,
        (x, y, w, h) = cv2.boundingRect(c)
        
#         ret, track_window = cv2.meanShift(dst,  (x, y, w, h), term_crit)
        
#         cv2.rectangle(img2, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        
        
#     if fr_number % 100 == 0:
#         track_window = (x, y, w, h)
        
    
    cv2.imshow('mask',flt_fgmask)
    cv2.imshow('frame',frame)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    fr_number += 1
cap.release()
cv2.destroyAllWindows()

(305, 231, 9, 9) (304, 230, 9, 9) (304, 230, 9, 9) 0.0 1.4142135623730951
(320, 218, 8, 7) (319, 217, 8, 7) (319, 218, 8, 7) 1.0 1.4142135623730951
(365, 202, 14, 8) (363, 202, 14, 8) (364, 202, 14, 8) 1.0 2.0
(325, 213, 8, 8) (325, 212, 8, 8) (325, 212, 8, 8) 0.0 1.0
(378, 188, 14, 11) (378, 188, 14, 11) (378, 187, 14, 11) 1.0 0.0


ValueError: too many values to unpack (expected 4)

In [23]:
print(old_cnts0)

[None]


In [22]:
len(old_cnts0)

1

In [39]:
len(cnts)

1

In [ ]:
for i in range(len(old_cnts)):
        ret, track_window = cv2.meanShift(dst, old_cnts[i], term_crit)
        print(old_cnts[i], track_window, 
              math.sqrt((old_cnts[i][0] - track_window[0])**2 + (old_cnts[i][1] - track_window[1])**2))

In [ ]:
math.sqrt

In [ ]:
cnts[0]

In [ ]:
x

In [ ]:
ret, track_window = cv2.meanShift(dst, cv2.boundingRect(cnts[0]), term_crit)

In [ ]:
ret, track_window = cv2.meanShift(dst, cv2.boundingRect(cnts[0]), term_crit)

In [ ]:
track_window

In [ ]:
x[:5],y[:5]

In [ ]:
cv2.boundingRect(cnts[0])

In [ ]:
x = []
y = []
for i in range(len(cnts[0])):
    x.append(cnts[0][i][0][0])
    y.append(cnts[0][i][0][1])
#     print(cnts[i][0][0][0])
plt.scatter(x,y)
# plt.imshow(fgmask)
plt.imshow(frame)
plt.show()

In [ ]:
x = []
y = []
print(len(cnts))
for c in cntrs:
#     print(center[0][0])
    x.append(center[0][0][0])
    y.append(center[0][0][1])
#     plt.plot(center[0][0])
plt.scatter(x,y)
# plt.imshow(fgmask)
plt.imshow(frame)
plt.show()

In [ ]:
print(type(fgmask))

In [ ]:
cap = cv2.VideoCapture(folder + '1-highway-test.mov')

In [ ]:
for i in range(100):
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 360), interpolation=cv2.INTER_AREA)
    fgmask = fgbg.apply(preprocessFrame(frame))

    flt_fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    flt_fgmask = cv2.threshold(flt_fgmask, 25, 255, cv2.THRESH_BINARY)[1]
    flt_fgmask = cv2.dilate(flt_fgmask, None, iterations=2)

In [ ]:
print(fgmask)

In [ ]:
cntrs = detect_vehicles(flt_fgmask)

In [ ]:
x = []
y = []
print(len(cnts))
for center in cntrs:
#     print(center[0][0])
    x.append(center[0][0][0])
    y.append(center[0][0][1])
#     plt.plot(center[0][0])
plt.scatter(x,y)
# plt.imshow(fgmask)
plt.imshow(frame)
plt.show()

### What are contours
cv2.findContours(), cv2.drawContours()

In [ ]:
def get_centroid(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)

    cx = x + x1
    cy = y + y1

    return (cx, cy)


In [ ]:
def detect_vehicles(fg_mask, min_contour_width=35, min_contour_height=35):

    matches = []

    # finding external contours
    im, contours, hierarchy = cv2.findContours(
        fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_L1)

    # filtering by with, height
    for (i, contour) in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        contour_valid = (w >= min_contour_width) and (
            h >= min_contour_height)

        if not contour_valid:
            continue
        
        # getting center of the bounding box
        centroid = get_centroid(x, y, w, h)

        matches.append(((x, y, w, h), centroid))

#     return matches
    return contours

In [ ]:
plt.imshow(cv2.dilate(thresh, None, iterations=2))
plt.show()

In [ ]:
plt.imshow(flt_fgmask)
plt.show()

In [ ]:
thresh = cv2.threshold(fgmask, 25, 255, cv2.THRESH_BINARY)[1]

In [ ]:
cnts = cv2.findContours(flt_fgmask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]

In [ ]:
for c in cnts:
    if cv2.contourArea(c) < 50:
        continue
 
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
    (x, y, w, h) = cv2.boundingRect(c)
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    text = "Occupied"
    print(cv2.contourArea(c))

In [ ]:
plt.imshow(frame)
plt.show()

In [ ]:
len(cnts)

In [ ]:
ret, frame = cap.read()
fgmask = fgbg.apply(frame)
car_boxes = detect_vehicles(fgmask)
print(car_boxes)

In [ ]:
len(car_boxes)

In [ ]:
bx = car_boxes[0]

In [ ]:
bx[0]

In [ ]:
plt.imshow(fgmask,cmap='gray')
for box in car_boxes:
    px = plt.plot(box[1])
    plt.setp(px, linewidth=10, color='r')
plt.show()

In [ ]:
for i, elem in enumerate(car_boxes):
    if elem.shape[0] > 35:
        print('lolololololololololololo', i)
        print(elem)

In [ ]:
print(car_boxes[110])

In [ ]:
plt.plot(car_boxes[110])
plt.show()